In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [26]:
#List of all teams and the components of the URL
url =['https://www.cbssports.com/nhl/teams/','/roster/']
teams = [
    ['BOS', 'boston-bruins'],
    ['BUF', 'buffalo-sabres'],
    ['DET', 'detroit-red-wings'],
    ['FLA', 'florida-panthers'],
    ['MON', 'montreal-canadiens'],
    ['OTT', 'ottawa-senators'],
    ['TB', 'tampa-bay-lightning'],
    ['TOR', 'toronto-maple-leafs'],
    ['ARI', 'arizona-coyotes'],
    ['CHI', 'chicago-blackhawks'],
    ['COL', 'colorado-avalanche'],
    ['DAL', 'dallas-stars'],
    ['MIN', 'minnesota-wild'],
    ['NSH', 'nashville-predators'],
    ['STL', 'st-louis-blues'],
    ['WPG', 'winnipeg-jets'],
    ['CAR', 'carolina-hurricanes'],
    ['CLB', 'columbus-blue-jackets'],
    ['NYI', 'new-york-islanders'],
    ['NYR', 'new-york-rangers'],
    ['NJ', 'new-jersey-devils'],
    ['PHI', 'philadelphia-flyers'],
    ['PIT', 'pittsburgh-penguins'],
    ['WAS', 'washington-capitals'],
    ['ANA', 'anaheim-ducks'],
    ['CGY', 'calgary-flames'],
    ['EDM', 'edmonton-oilers'],
    ['LA', 'los-angeles-kings'],
    ['SJ', 'san-jose-sharks'],
    ['SEA', 'seattle-kraken'],
    ['VAN', 'vancouver-canucks'],
    ['LV', 'vegas-golden-knights']]

In [27]:
#Makes an array of the team roster urls
pages = []
for team in teams:
    pages.append(f'{url[0]}{team[0]}/{team[1]}{url[1]}')

In [28]:
#takes beautiful soup item and team id to create an array of players for a whole team and returns the array
def getPlayersStats(players, teamID):
    playerStats = []
    for player in players:
        number = player.find_all('td', class_ = 'TableBase-bodyTd')[0].text.strip()
        name = ((player.find('span', class_ = 'CellPlayerName--long')).find('a').text).split(' ')
        fName = name[0]
        lName  = name[1]
        pos = player.find_all('td', class_ = 'TableBase-bodyTd')[2].text.strip()
        shot = player.find_all('td', class_ = 'TableBase-bodyTd')[3].text.strip()
        ht = player.find_all('td', class_ = 'TableBase-bodyTd')[4].text.strip()
        wt = player.find_all('td', class_ = 'TableBase-bodyTd')[5].text.strip()
        dob = player.find_all('td', class_ = 'TableBase-bodyTd')[6].text.strip()[:-5]
        birthplace = player.find_all('td', class_ = 'TableBase-bodyTd')[7].text.strip()
        playerID = f'{fName[:1]}{lName[:4]}{number}'
        team = teamID
        playerStats.append([playerID, fName, lName, number, pos, shot, ht, wt, dob, birthplace, team])
    return playerStats

In [35]:
#takes the list of pages itterates through and passes them to the get player stats function
#returns a list complete list of player stats
def get_teamPages(pages):
    count = 0
    playerList = []
    for page in pages:
        team = teams[count][0]
        url = requests.get(page)
        soup = BeautifulSoup(url.content, "html.parser")
        players = soup.find_all('tr', class_='TableBase-bodyTr')
        playerList = playerList + getPlayersStats(players,team)
        count+=1
    return playerList


In [1]:
df = pd.DataFrame(get_teamPages(pages))
df = df.rename(columns={0:'playerID', 1:'firstName', 2:'lastName', 3:'number', 4:'position', 5:'shot', 6:'height', 7:'weight', 8:'dob', 9: 'birthplace', 10:'teamID'})
df = df.set_index('playerID')

NameError: name 'pd' is not defined

In [54]:
#locates all players on the team with ID WPG - Winapeg Jets
df.loc[df['teamID'].str.contains('WPG')]

,firstName,lastName,number,position,shot,height,weight,dob,birhplace,teamID
playerID,,,,,,,,,,
MAppl22,Mason,Appleton,22,C,R,6-2,193,1/15/1996,"Green Bay, WI",WPG
MBarr36,Morgan,Barron,36,C,R,6-4,220,12/2/1998,"Halifax, NS",WPG
DGust19,David,Gustafsson,19,C,R,6-2,196,4/11/2000,"Tingsyrd, Sweden",WPG
RKupa15,Rasmus,Kupari,15,C,R,6-1,183,3/15/2000,"Kotka, Finland",WPG
ALowr17,Adam,Lowry,17,C,R,6-5,210,3/29/1993,"St. Louis, MO",WPG
VName7,Vladislav,Namestnikov,7,C,R,6-0,180,11/22/1992,"Zhukovskiy, Russia",WPG
CPerf91,Cole,Perfetti,91,C,R,5-11,177,1/1/2002,"Whitby, ON",WPG
MSche55,Mark,Scheifele,55,C,R,6-3,207,3/15/1993,"Kitchener, ON",WPG
GVila13,Gabriel,Vilardi,13,C,R,6-3,201,8/16/1999,"Kingston, ON",WPG


In [55]:
filePath = 'players.csv'

df.to_csv(filePath)